<a href="https://colab.research.google.com/github/prabirdeb/FriendLE/blob/main/FriendLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**All report files, evaluation files and student file must be set to view anyone with the link**

1. Error shall be managed through try else block


Note:
1. Student shall not change the subject names and headings
2. Student must share the file to view anyone with the link
3. No filter shall be selected

In [ ]:

# import re

# # Text processing
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem.snowball import SnowballStemmer
# import string

# # Data reading
# import pkgutil   # provides binary data
# from io import StringIO # for binary to high level data conversion

# # For showing image along with answer
# import urllib.request
# from PIL import Image

In [ ]:
# Writing helper functions

In [ ]:
# writing text pre-processing function
def text_process(text):
    text = text.lower()    # converting to lowercase
    import string
    text =[char for char in text if char not in string.punctuation] # removing punctuations
    text=''.join(text) 

    import nltk
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')

    tokens = nltk.word_tokenize(text)
    pos_tagged_tokens = nltk.pos_tag(tokens)

    list_of_verbs = []
    for i in range(len(pos_tagged_tokens)):
      if pos_tagged_tokens[i][1].startswith('V'):
        list_of_verbs.append(pos_tagged_tokens[i][0])
    
    nltk.download('stopwords')
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(list_of_verbs)

    text=[word for word in text.split() if word not in stopwords] # removing stopwords and verbs
    text=' '.join(text)

    return text

In [ ]:
# Writing main function

In [ ]:
# c=0

In [ ]:
def fill_blank(id, subject):
  try:
    # Importing libraries
    import numpy as np
    import pandas as pd
    # global c
    # Reading student data as pandas df
    gsheetid = "1g1uWDGjJ1aGRXtJVkISj9qwq-DJmnzTS"
    sheet_name = "Sheet1" # Student should not change the sheet name

    # Converting google sheet to csv
    gsheet_url = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(gsheetid, sheet_name)

    # Creating student data df
    student_data = pd.read_csv(gsheet_url)

    # Creating individual student df
    # Getting google sheet id
    gsheetid = student_data[(student_data["ID"]==id)]["Concept_link"].values[0].replace("https://docs.google.com/spreadsheets/d/","").split("/")[0]
    sheet_name = "Concepts" # Student should not change the sheet name

    # Converting google sheet to csv
    gsheet_url = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(gsheetid, sheet_name)

    # Creating individual student data df
    individual_student_data = pd.read_csv(gsheet_url)
    individual_student_data = individual_student_data[(individual_student_data["Class"]==student_data[(student_data["ID"]==id)]["Class"].values[0])]

    # Getting the document for the subject
    sub=subject.title() # Converting to title case
    subject_data=individual_student_data[(individual_student_data.Subjects==sub)]

    relevant_features=['Concept-1', 'Concept-2', 'Concept-3', 'Concept-4', 'Concept-5',
    'Concept-6', 'Concept-7', 'Concept-8', 'Concept-9', 'Concept-10',
    'Concept-11', 'Concept-12', 'Concept-13', 'Concept-14', 'Concept-15',
    'Concept-16', 'Concept-17', 'Concept-18', 'Concept-19', 'Concept-20']

    subject_data=subject_data[relevant_features]
    # Creating documents with all individual cell
    subject_data=pd.DataFrame(subject_data.values.flatten(), columns=['documents'])

    # Removing null value rows
    subject_data.dropna(inplace=True) 

    # There are many documents with only newline character. Removing those rows
    subject_data=subject_data[(subject_data['documents']!='\n')]
    subject_data=subject_data[(subject_data['documents']!='\n\n')]

    # Removing all the rows with no data and reseting index 
    subject_data=subject_data[(subject_data['documents']!='No data')].reset_index()

    subject_data.drop('index',axis=1, inplace=True)

    if subject_data.shape[0]!=0:
      # Random selection of concept
      index_lst=list(subject_data.index)
      import random
      random.shuffle(index_lst)
      original_text=subject_data.documents[index_lst[0]]

      # Generating answer and question
      # Random selection of question
      import random
      text=text_process(original_text)
      if len(text.split())>0:
        lst=text.split()
        random.shuffle(lst)
        ans=lst[0]
        
        ans_index=original_text.lower().split().index(ans)
        original_text=original_text.split()
        original_text[ans_index]="_____"
        ques=" ".join(original_text)
        # Question answer checking     
        result1=ques
        student_ans= "mango" #input("Write answer for the blank: ").lower()

        if student_ans==ans:
          # c=c+10
          result2="Awesome! Absolutely correct" # \nYour total score: {c}
        else:
          # c=c-10
          result2="Incorrect. Please revise the chapter.\nCorrect answer is:" # \nYour total score: {c}
      else:
        result2="Ask question again. Concept found is very poor in strength:("
      
    else:
      result2="You have no concept record for this subject"
  except:
    result2="Please provide correct id and subject name"
  
  return result1


In [ ]:
# Streamlit Project
import streamlit as st # All the text cell will be displayed after this import statement

st.title("Welcome to FriendLE Preparation")

id = st.number_input("Your ID")

subject = st.text_input("Subject")
subject=subject.title() # .title() is used to get the input question string

result = fill_blank(id, subject)

if(st.button('Ask Me')):   # display the ans when the submit button is clicked
  st.success(result)